In [1]:
import torch
import pandas as pd
import numpy as np
import sys 
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler

sys.path.append("./")
from prediction.api import *
from prediction.data_wrapper import get_loaders
from pickle import dump

In [2]:
args.data = './data/processed/processed_cleaned_df_2.csv'
args.input_size = [33]
args.hidden_size = 300
args.number_of_layers = 1
args.output_size = [1]
args.test_portion = 0.2 
args.train_window = 12
args.batch_size = 64
args.learning_rate = 0.001
args.weight_decay = 0.0001
args.gpu = 0 
args.epochs = 100
args.log_freq = 30
args.debug = False
args.seed = 1
args.samples = 20
args.dropout = 0.3

In [3]:
df = pd.read_csv(args.data).iloc[:,1:]
label_index = df.columns.get_loc('0411')
print(df.head())

      GPs  month      07    0304   0411  050103  131002   010305       01  \
0  A81001      1  7271.0  4071.0  758.0  1476.0   185.0  13032.0  69503.0   
1  A81001      2  8527.0  4071.0  758.0  1427.0   191.0  12740.0  78753.0   
2  A81001      3  6402.0  3928.0  758.0  1278.0   198.0  11574.0  65473.0   
3  A81001      4  7295.0  4836.0  758.0  1537.0   231.0  13019.0  74436.0   
4  A81001      5  6671.0  5153.0  758.0  1436.0   280.0  11824.0  65084.0   

       05  ...  030401       13           12     0106     1302       10  \
0  9767.0  ...  4310.0  59472.0   886.000000  19797.0  49930.0  15657.0   
1  9767.0  ...  4064.0  59472.0   994.666667  24304.0  49930.0  12903.0   
2  9767.0  ...  3928.0  59472.0  1103.333333  20416.0  44955.0  13236.0   
3  9767.0  ...  4836.0  58722.0  1212.000000  24713.0  45005.0  14082.0   
4  9767.0  ...  5152.0  57972.0   817.000000  20309.0  42955.0  12972.0   

    130201  050108    0603   010604  
0  41880.0   998.0  4034.0  10939.0  
1  48830.0

In [4]:
data = df.to_numpy()
gps = data[:,0]
features = data[:,1:]

In [5]:
print(features.shape, gps.shape)

(394458, 33) (394458,)


In [6]:
total_size = len(features)
train_size = int((1-args.test_portion) * total_size)

train_data = features[:train_size]
test_data = features[train_size:]

train_gps = gps[:train_size]
test_gps = gps[train_size:]

train_unique_gps = np.unique(train_gps)
test_unique_gps = np.unique(test_gps)

scaler = MinMaxScaler(feature_range=(-1, 1))

train_data_normalized = scaler.fit_transform(train_data)
test_data_normalized = scaler.transform(test_data)

dump(scaler, open("model/scaler.pck", "wb"))


train_data_normalized = torch.FloatTensor(train_data_normalized)
test_data_normalized = torch.FloatTensor(test_data_normalized)

In [7]:
def create_inout_sequences(input_data,_gps,_unique_gps,_label_index, tw):
    X = []
    Y = []
    for i in tqdm(range(len(_unique_gps))):
        for j in range(len(_gps[_gps == _unique_gps[i]]) - tw):
            seq = input_data[j:j+tw]
            label = input_data[j+tw:j+tw+1,_label_index]
            seq = np.array(seq, dtype=np.float32)
            label = np.array(label, dtype = np.float32)
            X.append(torch.from_numpy(seq).float())
            Y.append(torch.from_numpy(label).float())

    return X, Y
X_train, Y_train = create_inout_sequences(train_data_normalized, train_gps, train_unique_gps, label_index, args.train_window)
X_test, Y_test = create_inout_sequences(test_data_normalized, test_gps, test_unique_gps, label_index, args.train_window)


100%|██████████| 1361/1361 [00:02<00:00, 508.13it/s]


In [8]:
X_train = torch.stack(X_train)
X_test = torch.stack(X_test)
Y_train = torch.stack(Y_train)
Y_test = torch.stack(Y_test)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

torch.Size([250274, 12, 33]) torch.Size([250274, 1])
torch.Size([62560, 12, 33]) torch.Size([62560, 1])


In [9]:
def get_loaders(X, Y):
    dataset = torch.utils.data.TensorDataset(X,Y)
    return torch.utils.data.DataLoader(dataset,batch_size = args.batch_size,shuffle=True)

In [10]:
train_loader = get_loaders(X_train, Y_train)
test_loader = get_loaders(X_test, Y_test)

In [11]:
model, scaler = get_model(train_loader, test_loader)

02/29 11:54:26 PM | # START #
02/29 11:54:26 PM | Args = Namespace(batch_size=64, data='./data/processed/processed_cleaned_df_2.csv', debug=False, dropout=0.3, epochs=100, f='/home/killko/.local/share/jupyter/runtime/kernel-fea49255-0cf2-4839-a856-a145aadab768.json', gpu=0, hidden_size=300, input_size=[33], layers=8, learning_rate=0.001, log_freq=30, model_path='./model/drugs-EXP-20200229-235351/drugs-EXP-20200229-235351.model', num_of_layers=1, number_of_layers=1, output_size=[1], samples=20, save='./model/drugs-EXP-20200229-235351', seed=1, test_portion=0.2, train_window=12, weight_decay=0.0001)
INFO:net:Args = Namespace(batch_size=64, data='./data/processed/processed_cleaned_df_2.csv', debug=False, dropout=0.3, epochs=100, f='/home/killko/.local/share/jupyter/runtime/kernel-fea49255-0cf2-4839-a856-a145aadab768.json', gpu=0, hidden_size=300, input_size=[33], layers=8, learning_rate=0.001, log_freq=30, model_path='./model/drugs-EXP-20200229-235351/drugs-EXP-20200229-235351.model', num

02/29 11:54:30 PM | #### Train: [  1/100] Step 1350/3910 MSE 0.0566+-0.00000
INFO:net:#### Train: [  1/100] Step 1350/3910 MSE 0.0566+-0.00000
02/29 11:54:31 PM | #### Train: [  1/100] Step 1380/3910 MSE 0.0558+-0.00000
INFO:net:#### Train: [  1/100] Step 1380/3910 MSE 0.0558+-0.00000
02/29 11:54:31 PM | #### Train: [  1/100] Step 1410/3910 MSE 0.0550+-0.00000
INFO:net:#### Train: [  1/100] Step 1410/3910 MSE 0.0550+-0.00000
02/29 11:54:31 PM | #### Train: [  1/100] Step 1440/3910 MSE 0.0542+-0.00000
INFO:net:#### Train: [  1/100] Step 1440/3910 MSE 0.0542+-0.00000
02/29 11:54:31 PM | #### Train: [  1/100] Step 1470/3910 MSE 0.0534+-0.00000
INFO:net:#### Train: [  1/100] Step 1470/3910 MSE 0.0534+-0.00000
02/29 11:54:31 PM | #### Train: [  1/100] Step 1500/3910 MSE 0.0527+-0.00000
INFO:net:#### Train: [  1/100] Step 1500/3910 MSE 0.0527+-0.00000
02/29 11:54:31 PM | #### Train: [  1/100] Step 1530/3910 MSE 0.0520+-0.00000
INFO:net:#### Train: [  1/100] Step 1530/3910 MSE 0.0520+-0.00000

INFO:net:#### Train: [  1/100] Step 3060/3910 MSE 0.0324+-0.00000
02/29 11:54:35 PM | #### Train: [  1/100] Step 3090/3910 MSE 0.0322+-0.00000
INFO:net:#### Train: [  1/100] Step 3090/3910 MSE 0.0322+-0.00000
02/29 11:54:35 PM | #### Train: [  1/100] Step 3120/3910 MSE 0.0319+-0.00000
INFO:net:#### Train: [  1/100] Step 3120/3910 MSE 0.0319+-0.00000
02/29 11:54:35 PM | #### Train: [  1/100] Step 3150/3910 MSE 0.0317+-0.00000
INFO:net:#### Train: [  1/100] Step 3150/3910 MSE 0.0317+-0.00000
02/29 11:54:35 PM | #### Train: [  1/100] Step 3180/3910 MSE 0.0314+-0.00000
INFO:net:#### Train: [  1/100] Step 3180/3910 MSE 0.0314+-0.00000
02/29 11:54:36 PM | #### Train: [  1/100] Step 3210/3910 MSE 0.0312+-0.00000
INFO:net:#### Train: [  1/100] Step 3210/3910 MSE 0.0312+-0.00000
02/29 11:54:36 PM | #### Train: [  1/100] Step 3240/3910 MSE 0.0309+-0.00000
INFO:net:#### Train: [  1/100] Step 3240/3910 MSE 0.0309+-0.00000
02/29 11:54:36 PM | #### Train: [  1/100] Step 3270/3910 MSE 0.0307+-0.00000

02/29 11:54:50 PM | #### Test: [  1/100] Step 870/977 MSE 0.5884+-0.03061
INFO:net:#### Test: [  1/100] Step 870/977 MSE 0.5884+-0.03061
02/29 11:54:50 PM | #### Test: [  1/100] Step 900/977 MSE 0.5883+-0.03061
INFO:net:#### Test: [  1/100] Step 900/977 MSE 0.5883+-0.03061
02/29 11:54:51 PM | #### Test: [  1/100] Step 930/977 MSE 0.5880+-0.03060
INFO:net:#### Test: [  1/100] Step 930/977 MSE 0.5880+-0.03060
02/29 11:54:51 PM | #### Test: [  1/100] Step 960/977 MSE 0.5879+-0.03060
INFO:net:#### Test: [  1/100] Step 960/977 MSE 0.5879+-0.03060
02/29 11:54:51 PM | #### Test: [  1/100] Step 977/977 MSE 0.5879+-0.03061
INFO:net:#### Test: [  1/100] Step 977/977 MSE 0.5879+-0.03061
02/29 11:54:51 PM | Test: [  1/100] Final MSE 0.5879+-0.03061 Time 14.01
INFO:net:Test: [  1/100] Final MSE 0.5879+-0.03061 Time 14.01
02/29 11:54:51 PM | Best top1 acc by now. Save model
INFO:net:Best top1 acc by now. Save model
02/29 11:54:51 PM | #### Training weights for epoch 1 #### 
INFO:net:#### Training we

02/29 11:54:56 PM | #### Train: [  2/100] Step 1560/3910 MSE 0.0007+-0.00000
INFO:net:#### Train: [  2/100] Step 1560/3910 MSE 0.0007+-0.00000
02/29 11:54:56 PM | #### Train: [  2/100] Step 1590/3910 MSE 0.0007+-0.00000
INFO:net:#### Train: [  2/100] Step 1590/3910 MSE 0.0007+-0.00000
02/29 11:54:56 PM | #### Train: [  2/100] Step 1620/3910 MSE 0.0007+-0.00000
INFO:net:#### Train: [  2/100] Step 1620/3910 MSE 0.0007+-0.00000
02/29 11:54:56 PM | #### Train: [  2/100] Step 1650/3910 MSE 0.0007+-0.00000
INFO:net:#### Train: [  2/100] Step 1650/3910 MSE 0.0007+-0.00000
02/29 11:54:56 PM | #### Train: [  2/100] Step 1680/3910 MSE 0.0007+-0.00000
INFO:net:#### Train: [  2/100] Step 1680/3910 MSE 0.0007+-0.00000
02/29 11:54:56 PM | #### Train: [  2/100] Step 1710/3910 MSE 0.0007+-0.00000
INFO:net:#### Train: [  2/100] Step 1710/3910 MSE 0.0007+-0.00000
02/29 11:54:56 PM | #### Train: [  2/100] Step 1740/3910 MSE 0.0007+-0.00000
INFO:net:#### Train: [  2/100] Step 1740/3910 MSE 0.0007+-0.00000

...

INFO:net:#### Test: [  6/100] Step 210/977 MSE 0.4748+-0.03916
02/29 11:56:43 PM | #### Test: [  6/100] Step 240/977 MSE 0.4776+-0.03934
INFO:net:#### Test: [  6/100] Step 240/977 MSE 0.4776+-0.03934
02/29 11:56:44 PM | #### Test: [  6/100] Step 270/977 MSE 0.4806+-0.03935
INFO:net:#### Test: [  6/100] Step 270/977 MSE 0.4806+-0.03935
02/29 11:56:44 PM | #### Test: [  6/100] Step 300/977 MSE 0.4812+-0.03939
INFO:net:#### Test: [  6/100] Step 300/977 MSE 0.4812+-0.03939
02/29 11:56:44 PM | #### Test: [  6/100] Step 330/977 MSE 0.4809+-0.03946
INFO:net:#### Test: [  6/100] Step 330/977 MSE 0.4809+-0.03946
02/29 11:56:45 PM | #### Test: [  6/100] Step 360/977 MSE 0.4816+-0.03941
INFO:net:#### Test: [  6/100] Step 360/977 MSE 0.4816+-0.03941
02/29 11:56:45 PM | #### Test: [  6/100] Step 390/977 MSE 0.4822+-0.03937
INFO:net:#### Test: [  6/100] Step 390/977 MSE 0.4822+-0.03937
02/29 11:56:46 PM | #### Test: [  6/100] Step 420/977 MSE 0.4829+-0.03934
INFO:net:#### Test: [  6/100] Step 420/97

02/29 11:56:56 PM | #### Train: [  7/100] Step 930/3910 MSE 0.0001+-0.00000
INFO:net:#### Train: [  7/100] Step 930/3910 MSE 0.0001+-0.00000
02/29 11:56:56 PM | #### Train: [  7/100] Step 960/3910 MSE 0.0001+-0.00000
INFO:net:#### Train: [  7/100] Step 960/3910 MSE 0.0001+-0.00000
02/29 11:56:56 PM | #### Train: [  7/100] Step 990/3910 MSE 0.0001+-0.00000
INFO:net:#### Train: [  7/100] Step 990/3910 MSE 0.0001+-0.00000
02/29 11:56:56 PM | #### Train: [  7/100] Step 1020/3910 MSE 0.0001+-0.00000
INFO:net:#### Train: [  7/100] Step 1020/3910 MSE 0.0001+-0.00000
02/29 11:56:57 PM | #### Train: [  7/100] Step 1050/3910 MSE 0.0001+-0.00000
INFO:net:#### Train: [  7/100] Step 1050/3910 MSE 0.0001+-0.00000
02/29 11:56:57 PM | #### Train: [  7/100] Step 1080/3910 MSE 0.0001+-0.00000
INFO:net:#### Train: [  7/100] Step 1080/3910 MSE 0.0001+-0.00000
02/29 11:56:57 PM | #### Train: [  7/100] Step 1110/3910 MSE 0.0001+-0.00000
INFO:net:#### Train: [  7/100] Step 1110/3910 MSE 0.0001+-0.00000
02/29

INFO:net:#### Train: [  7/100] Step 2640/3910 MSE 0.0005+-0.00000
02/29 11:57:01 PM | #### Train: [  7/100] Step 2670/3910 MSE 0.0005+-0.00000
INFO:net:#### Train: [  7/100] Step 2670/3910 MSE 0.0005+-0.00000
02/29 11:57:01 PM | #### Train: [  7/100] Step 2700/3910 MSE 0.0005+-0.00000
INFO:net:#### Train: [  7/100] Step 2700/3910 MSE 0.0005+-0.00000
02/29 11:57:01 PM | #### Train: [  7/100] Step 2730/3910 MSE 0.0005+-0.00000
INFO:net:#### Train: [  7/100] Step 2730/3910 MSE 0.0005+-0.00000
02/29 11:57:01 PM | #### Train: [  7/100] Step 2760/3910 MSE 0.0005+-0.00000
INFO:net:#### Train: [  7/100] Step 2760/3910 MSE 0.0005+-0.00000
02/29 11:57:01 PM | #### Train: [  7/100] Step 2790/3910 MSE 0.0005+-0.00000
INFO:net:#### Train: [  7/100] Step 2790/3910 MSE 0.0005+-0.00000
02/29 11:57:01 PM | #### Train: [  7/100] Step 2820/3910 MSE 0.0005+-0.00000
INFO:net:#### Train: [  7/100] Step 2820/3910 MSE 0.0005+-0.00000
02/29 11:57:01 PM | #### Train: [  7/100] Step 2850/3910 MSE 0.0005+-0.00000

INFO:net:#### Test: [  7/100] Step 420/977 MSE 0.5359+-0.03844
02/29 11:57:11 PM | #### Test: [  7/100] Step 450/977 MSE 0.5354+-0.03846
INFO:net:#### Test: [  7/100] Step 450/977 MSE 0.5354+-0.03846
02/29 11:57:11 PM | #### Test: [  7/100] Step 480/977 MSE 0.5353+-0.03846
INFO:net:#### Test: [  7/100] Step 480/977 MSE 0.5353+-0.03846


KeyboardInterrupt: 